# Jupyter Widgets

## Upload File and Convert

In [2]:
from ipywidgets import FileUpload, Button, Output
from IPython.display import display
import subprocess

output = Output()
fileInput = FileUpload()
convertButton = Button(description='Convert')

def convert(b):
    with output:
        nbName = list(fileInput.value.keys())[0]
        with open(nbName, 'w+b') as f:
            f.write(fileInput.data[0]) 
        pdfName = nbName.replace('ipynb', 'pdf')
        subprocess.run(['jupyter', 'nbconvert', '--to', 'pdf', nbName])

convertButton.on_click(convert)

display(fileInput, convertButton, output) 

FileUpload(value={}, description='Upload')

Button(description='Convert', style=ButtonStyle())

Output()

## [Download File](https://stackoverflow.com/questions/61708701/how-to-download-a-file-using-ipywidget-button)

In [9]:
with open('__dash1.pdf', 'rb') as f:
    pdf = f.read().decode('ASCII-85')


LookupError: unknown encoding: ASCII-85

In [12]:
from ipywidgets import HTML


display(HTML('''
    <a href="__dash1.pdf" download>
    <button class="p-Widget jupyter-widgets jupyter-button widget-button">Download File</button>
    </a>
'''))

HTML(value='\n    <a download="__dash1.pdf" href="pdf" download>\n    <button class="p-Widget jupyter-widgets …

In [3]:
import base64

res = 'res'
b64 = base64.b64encode(res.encode())
payload = b64.decode()

type(b64), type(payload)

(bytes, str)

In [1]:
from ipywidgets import HTML
from IPython.display import display
import base64

#FILE

res = 'res'
filename = 'res.txt'
b64 = base64.b64encode(res.encode())
payload = b64.decode()

#BUTTONS
html_buttons = '''<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<a download="{filename}" href="data:text/csv;base64,{payload}" download>
<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning">Download File</button>
</a>
</body>
</html>
'''

html_button = html_buttons.format(payload=payload, filename=filename)

display(HTML(html_button))

SyntaxError: invalid syntax (<ipython-input-1-4a08e38628c2>, line 7)